In [1]:
USE {
    repositories {
        mavenCentral()
    }

    dependencies {
        implementation("org.apache.commons:commons-math3:3.6.1")
    }
}

In [2]:
%use kandy
%use dataframe

In [3]:
import kotlin.random.Random

// Prepare the measurement data 
fun getRandomMeasure(
    trueValue: Double,
    range: Double,
    steps: Int
): Pair<DoubleArray, DoubleArray> {

    val measureVal = Random.nextDouble(trueValue - range, trueValue + range) 
    val diffVal = trueValue - measureVal

    val incrementor = steps * 20
    return Pair(doubleArrayOf(measureVal + incrementor), doubleArrayOf(diffVal*diffVal))
}

In [4]:
import org.apache.commons.math3.linear.ArrayRealVector
import org.apache.commons.math3.linear.Array2DRowRealMatrix
import org.apache.commons.math3.linear.RealMatrix
import org.apache.commons.math3.linear.RealVector
import org.kalman.wrapper.KalmanFilterProcess


val kfProcess = KalmanFilterProcess(
    debugEnabled = true,
    states = ArrayRealVector(
        doubleArrayOf(
            50.0 //initial estimation
        )
    ),
    stateTransitionMatrix = Array2DRowRealMatrix(
        arrayOf(
            doubleArrayOf(1.0) // assuming the state is still
        )
    ),
    covarianceMatrix = Array2DRowRealMatrix(
        arrayOf(
            doubleArrayOf(400.0 * 400.0) // initial co-variance
        )
    ),
    processMatrix = Array2DRowRealMatrix(
        arrayOf(
            doubleArrayOf(25*25.0) // assuming the process remain still
        )
    )
)



repeat(100){step ->
    kfProcess.predict()
    
    val (measureVal, measureCovariances) = getRandomMeasure(trueValue = 400.0, range = 200.0, steps = step + 1)
    kfProcess.update(
        y = ArrayRealVector(measureVal),
        r = Array2DRowRealMatrix(
            arrayOf(
                measureCovariances
            )
        )
    )
}

val debug = kfProcess.debug()

val plotData = dataFrameOf(
    "data" to debug.map { it.x.maxValue } + debug.map { it.y.maxValue },
    "type" to List(debug.size) { "estimate" } + List(debug.size) { "measurement" },
    "index" to List(debug.size) { it } +  List(debug.size) { it }
)

plotData.plot {
    line {
        x("index")
        y("data")
        color("type"){
            scale = categorical("measurement" to Color.BLUE, "estimate" to Color.RED )
        }
    }
    layout.size = 1000 to 450
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="yIFnJ0"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"data":[486.5798653168425,415.03661059667917,453.2105307726242,488.64302964909564,498.5630988038641,507.5381109146,514.426623722268,508.5329054630808,512.8326895612489,513.4622865285256,621.4455770406239,625.5620940039161,650.1585711487066,655.138468865402,666.1760816051228,705.6919393678787,702.0655554010395,732.6587509615665,726.0249171196776,742.5121963026206,824.5952115967211,814.1936135235006,864.5089038531264,873.6040103315682,881.0660253101687,872.296248214226,901.723948586137,899.1603052580978,941.3576628057463,938.2445799458256,937.4845509385269,942.9299392496985,1001.1735414893599,1015.626361833222,1056.9032203754857,1128.0506526278155,1122.426653080416,1145.5022792926788,1136.2911098650934,1155.1184890148097,1145.447934223907,1179.8235852382816,1181.1576243861289,1198.5133244632439,1254.647410676095,1278.3147992593538,1301.3012365793722,1290.9293043545895,1319.4280129653023,1329.3877253367496,1324.8371929106818,1331.0610624609049,1395.9028308362304,1414.2010658676759,1444.5504694007468,1435.876996307883,1502.5765244605625,1546.1805350770412,1562.7269087663701,1553.6596531971077,1546.8737047406546,1621.4907962963814,1656.0012184153336,1650.5976475933587,1645.237260694546,1647.597709766296,1643.1446083882,1675.5919803065715,1771.549231138666,1779.863857242922,1790.7915227198978,1809.7106774771632,1824.4321958712321,1853.4634565178287,1843.3900835319143,1858.4344402483869,1892.3664898604802,1885.7537443809344,1908.4023125622916,1901.2099624729224,1931.229970592875,1945.7226951705227,1947.7377023162956,1949.458857079193,1948.739326304522,1967.0511609617722,1981.6374249557837,2034.4172580358977,2109.590966621003,2164.658928452261,2182.118111984412,2176.277511671648,2238.067582420224,2244.58396042473,2269.5142677144904,2274.9709283009856,2292.600062355861,2286.377036449148,2317.168808262303,2310.128208543773,507.28969378698366,367.7186896450418,565.1285349926945,567.3794710035517,498.5698740697936,600.659708669403,736.3831332485993,494.2785204527301,520.8096158127539,515.8684471640178,621.5732825714887,805.708738477065,651.5699734215411,840.5280918368052,845.1284961847475,708.5126623656641,583.7888875802819,823.246298375245,613.3995241484322,974.8123555737146,846.7378004617397,765.1770755234331,865.4348483456508,957.8174359041332,1079.9907419860542,790.7948600988082,1044.3544206565655,894.7759560871516,954.1778890532371,921.2605131891718,933.933417788395,960.6754269395278,1127.5397099550041,1270.9656047874873,1191.7987370580367,1144.559167276786,961.311036611282,1246.1657335916216,1022.3020228899538,1367.3426987808602,1083.0873939411345,1198.5883214583275,1185.4343142113053,1446.9395410271309,1371.2693365281548,1447.5604223341377,1484.5626546061849,1242.245491140428,1336.8039700932254,1345.7916465021162,1237.2291507722289,1352.1700863476108,1530.2434491056645,1649.32361563149,1627.4423128406627,1337.4579876389957,1515.0593418004896,1596.364087359168,1669.404434985939,1515.88091682406,1462.801421002619,1686.3367739247037,1656.2716665631613,1616.6550922523234,1594.6705631887453,1653.7364980423888,1585.5551869690855,1881.8999254987623,1814.3404795394988,1991.477644120374,1794.3628597456188,1921.2853850240963,1995.7069432886988,1972.351526204456,1795.957625808829,1872.1162407404497,2024.2350413259958,1839.6176123339178,2136.981410697014,1809.9668922723217,2186.6312050353054,1967.3174836827407,1955.7162297738214,1958.4790961314548,1942.7946116660637,2021.2610616881555,2034.3664080386416,2098.986948132418,2244.154693777993,2177.6991365428466,2305.0362686852477,2089.5453149196005,2244.915443656886,2436.21793710796,2379.28054556327

In [5]:
val debugDf = dataFrameOf(
    "kalman_gain" to debug.map { it.k.getEntry(0 , 0) },
    "covariance" to debug.map { it.p.getEntry(0,0) },
    "index" to List(debug.size) { it }
)


debugDf.plot {
    line {
        x("index")
        y("kalman_gain")
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Sqs0bS"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"index":[0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0,40.0,41.0,42.0,43.0,44.0,45.0,46.0,47.0,48.0,49.0,50.0,51.0,52.0,53.0,54.0,55.0,56.0,57.0,58.0,59.0,60.0,61.0,62.0,63.0,64.0,65.0,66.0,67.0,68.0,69.0,70.0,71.0,72.0,73.0,74.0,75.0,76.0,77.0,78.0,79.0,80.0,81.0,82.0,83.0,84.0,85.0,86.0,87.0,88.0,89.0,90.0,91.0,92.0,93.0,94.0,95.0,96.0,97.0,98.0,99.0],
"kalman_gain":[0.9547117970260044,0.6019059992954173,0.25433693604476504,0.3103514739192878,0.9993174803953411,0.08790705315833702,0.030101213202768557,0.2925197563434166,0.350239109313404,0.2073936887129776,0.9988187553945578,0.022340422964641726,0.9457317434393339,0.026159112544357852,0.058095747947696924,0.9333738590336419,0.02974809829337018,0.2524592176088494,0.055625329956929065,0.06627054519068501,0.7875513503264142,0.1750576298857785,0.9819296998649971,0.0974734708447671,0.036155497588346366,0.09714926217479543,0.17103343569322704,0.3689761220112991,0.7669794752305459,0.1549017109049707,0.17629330086524053,0.23480735746836623,0.31549577260802836,0.05357022058833272,0.23429813266461427,0.8116669178468978,0.03372923413019003,0.1864861619864171,0.07476583001154906,0.08148560777298983,0.13425527934861267,0.6468837131865773,0.23776574385632263,0.06530053021085583,0.3249327507731506,0.12268425224324199,0.11145055196508068,0.1756295200016745,0.6212297823177616,0.37778223390715715,0.04937720063022395,0.2277062091719166,0.3255396697850113,0.07220494974880949,0.14232426401499884,0.08099049485247446,0.8423535289050931,0.46492316538408623,0.13427893210296143,0.19355456450636443,0.07468721647508354,0.5350311877709706,0.9922242271181451,0.13733425241376915,0.09584599225720723,0.2777248093147018,0.07177498878714722,0.13590219606856302,0.6915912702666728,0.0378060569415718,0.7536839499781609,0.14498118473313637,0.07914953738364672,0.19626414330836076,0.1751713323973918,0.5237162982594308,0.2046557695904269,0.12536277148676603,0.09015156854923,0.0730666874604907,0.10517790424583151,0.4015994269695943,0.20163107706734784,0.1602357005539025,0.10796882918709877,0.2525013302616423,0.2166858865920088,0.4497660633922638,0.35841817327042763,0.8085368010100776,0.1243732321981144,0.06309197191778669,0.9002321159947695,0.03288602745513649,0.18508492449246036,0.5263207926281336,0.09671049633877007,0.12346362282147376,0.1463357221908652,0.10235331380263732]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"index",
"y":"kalman_gain"
},
"stat":"identity",
"sampling":"none",
"position":"identity",
"geom":"line",
"data":{
}
}]
};
 var plotContainer = document.getElementById("Sqs0bS");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M24.4075135808963 29.653619820775816 L24.4075135808963 29.653619820775816 L29.338324405319796 141.60076992490198 L34.26913522974329 251.8862764147461 L39.19994605416679 234.11256657270738 L44.130756878590276 15.5 L49.06156770301378 304.6953606223887 L53.99237852743727 323.0374601117591 L58.92318935186076 239.77066509864147 L63.85400017628426 221.45600838832735 L68.78481100070775 266.78161767396125 L73.71562182513125 15.658248086858293 L78.64643264955475 325.5 L83.57724347397823 32.50303843372552 L88.50805429840173 324.2883095506611 L93.43886512282523 314.1546458174804 L98.36967594724874 36.424